In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf 
from tensorflow import keras

In [42]:
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
data = pd.read_csv('insurance_data.csv')
data.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [3]:
X = data[['age', 'affordibility']]
y = data['bought_insurance']

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age']/100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age']/100

In [10]:
import math
def sigmoid(x):
    return 1/(1+np.exp(-x))
sigmoid(10)

np.float64(0.9999546021312976)

In [11]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [44]:
class myNN:
    def __init__(self):
        self.w1 = 1
        self.w2 = 1
        self.w3 = 0

    def predict(self, X_test):
        weigthed_sum = self.w1 * X_test['age'] + self.w2 * X_test['affordibility'] + self.bias
        return sigmoid(weigthed_sum)

    def fit(self, X, y, epochs, loss_thresold):
        self.w1, self.w2, self.bias = self.gradient_descent(X['age'], X['affordibility'], y, epochs, loss_thresold)

    def gradient_descent(self, age, affordability, y_true, epochs, loss_thresold):
        w1 = w2 = 1
        bias = 0
        rate = 0.5
        n = len(age)
        for i in range(epochs):
            weighted_sum = w1 * age + w2 * affordability + bias
            y_predicted = sigmoid(weighted_sum)
            loss = log_loss(y_true, y_predicted)

            w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true)) 
            w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true)) 

            bias_d = np.mean(y_predicted-y_true)
            w1 = w1 - rate * w1d
            w2 = w2 - rate * w2d
            bias = bias - rate * bias_d

            print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
           

            if loss<=loss_thresold:
                print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
                print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
                break

        return w1, w2, bias

In [45]:
customModel = myNN()
customModel.fit(X_train_scaled, y_train, epochs=4000, loss_thresold=0.5281)

Epoch:0, w1:0.9736899318847281, w2:0.931388810977659, bias:-0.11748951666770448, loss:0.7428288579142563
Epoch:1, w1:0.9536535852311094, w2:0.8740290167758512, bias:-0.21881533456146035, loss:0.7072146449948487
Epoch:2, w1:0.9393731039296969, w2:0.8271852202997496, bias:-0.3053620401943441, loss:0.6814881914786812
Epoch:3, w1:0.9301932588998061, w2:0.7897792032048467, bias:-0.37884372361582785, loss:0.6633428084673968
Epoch:4, w1:0.9254091137248938, w2:0.7605726653866934, bias:-0.44108236820018304, loss:0.650742850709519
Epoch:5, w1:0.9243325693598607, w2:0.738313053647322, bias:-0.49384257986251556, loss:0.6420508089402462
Epoch:6, w1:0.926333296357235, w2:0.7218280753843739, bias:-0.5387319906498417, loss:0.6360356979531208
Epoch:7, w1:0.930858097563688, w2:0.7100747303660235, bias:-0.5771558825717441, loss:0.631816485354411
Epoch:8, w1:0.9374354910317362, w2:0.7021560855322683, bias:-0.6103083840841516, loss:0.6287844495353145
Epoch:9, w1:0.9456716791005845, w2:0.6973185496313956, b

In [30]:
customModel.predict(X_test_scaled)

9     0.759932
25    0.712346
8     0.766272
21    0.229190
0     0.446332
12    0.235445
dtype: float64

* comparing with the keras model

In [28]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1, input_shape=(X_train_scaled.shape[1],), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.fit(X_train_scaled, y_train, epochs=3500)

d:\AIML Internship\Day 41-43 Nueral Nework\tf_env\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/3500
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.5000 - loss: 0.7428
Epoch 2/3500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.5000 - loss: 0.7424
Epoch 3/3500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.5000 - loss: 0.7420
Epoch 4/3500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.5000 - loss: 0.7416
Epoch 5/3500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.5000 - loss: 0.7411
Epoch 6/3500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.5000 - loss: 0.7407
Epoch 7/3500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.5000 - loss: 0.7403
Epoch 8/3500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.5000 - loss: 0.7399
Epoch 9/3500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.5000 - loss: 0.7395
Epoch 10/3500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.5000 - loss: 0.7390
Epoch 11/3500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.5000 - loss: 0.7386
Epoch 12/3500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0

In [31]:
model.evaluate(X_test_scaled, y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step - accuracy: 1.0000 - loss: 0.3420


[0.3420158326625824, 1.0]

In [38]:
y_pred = model.predict(X_test_scaled)
y_pred

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step


array([[0.76283336],
       [0.7143475 ],
       [0.7692766 ],
       [0.2556099 ],
       [0.44177577],
       [0.26251101]], dtype=float32)

In [50]:
y_pred_nn = customModel.predict(X_test_scaled)
y_pred_nn

9     0.759932
25    0.712346
8     0.766272
21    0.229190
0     0.446332
12    0.235445
dtype: float64

* We can see prediction of both custom Model and Pre seed Model is almost same so we can consider our custom model is kinda perfect

In [34]:
coef, intercept = model.get_weights()
coef, intercept

(array([[3.5954738 ],
        [0.97877353]], dtype=float32),
 array([-2.003736], dtype=float32))

coef, intercept from custom NN: Epoch:203, w1:3.507208079655616, w2:1.1376738327577196, bias:-2.1247631025315683, loss:0.5280465392885548

* we can see weigth and bias are also same

In [48]:
r2_score(y_test,y_pred)

0.6530047655105591

In [51]:
r2_score(y_test, y_pred_nn)

0.6652127658990725